In [1]:
import os

os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'password'

In [2]:
import pyspark
import os
from pyspark.sql import SparkSession
from pyspark import SparkContext

# Set the required configuration
spark_conf = pyspark.SparkConf()
spark_conf.set("spark.sql.warehouse.dir", "s3a://warehouse")
spark_conf.set("hive.metastore.uris", "thrift://hive-metastore:9083")
spark_conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark_conf.set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1,org.apache.spark:spark-hadoop-cloud_2.13:3.3.1,org.apache.hadoop:hadoop-common:3.3.1,com.amazonaws:aws-java-sdk-bundle:1.12.172")
# spark_conf.set("spark.jars.packages", "org.apache.spark:spark-hadoop-cloud_2.13")

def load_config(spark_context: SparkContext):
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID", "admin"))
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.secret.key",
                                                 os.getenv("AWS_SECRET_ACCESS_KEY", "password"))
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.endpoint", os.getenv("ENDPOINT", "http://minio:9000"))
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
    spark_context._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")
    spark_context._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.s3a.impl", "org.apache.hadoop.fs.s3a.S3A")


# Create the SparkSession with the configuration and enable Hive support
spark = (
    SparkSession.builder
    .master("local[*]")
    .config(conf=spark_conf)
    .enableHiveSupport()
    .getOrCreate()
)

load_config(spark.sparkContext)
spark.sql("CREATE TABLE IF NOT EXISTS default.sample_table (id INT, name STRING)")

DataFrame[]

In [3]:
spark.version

'3.3.1'

In [4]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



## Create Table and Insert Records

In [5]:
spark.sql(
f"""
CREATE TABLE IF NOT EXISTS default.test_table (
  c1 STRING,
  city STRING)
USING parquet
PARTITIONED BY (city)
LOCATION 's3a://warehouse/default/test_table'
"""
)

spark.sql(
f"""
INSERT INTO default.test_table PARTITION (city='PARK')
    (c1)
    VALUES 
    ('testing1234');
""")

DataFrame[]

## Drop Table

In [6]:
spark.sql(f'drop table if exists default.test_table')

DataFrame[]

## Recreate Table

In [7]:
spark.sql(
f"""
CREATE TABLE IF NOT EXISTS default.test_table (
  c1 STRING,
  city STRING)
USING parquet
PARTITIONED BY (city)
LOCATION 's3a://warehouse/default/test_table'
"""
)

spark.sql(
f"""
INSERT INTO default.test_table PARTITION (city='PARK')
    (c1)
    VALUES 
    ('testing1234');
""")

Py4JJavaError: An error occurred while calling o37.sql.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org.apac